<a href="https://colab.research.google.com/github/blennyblex/UmojaHackAfrica/blob/main/UmojaHackAfrica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np

from datetime import date, time, datetime


import warnings
warnings.simplefilter(action='ignore')


from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import 

from sklearn.metrics import accuracy_score
import math

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pd.options.display.max_columns = 60

In [5]:
# Load files
train = pd.read_csv('/content/drive/My Drive/Umojack/Train.csv')
test = pd.read_csv('/content/drive/My Drive/Umojack/Test.csv')
riders = pd.read_csv('/content/drive/My Drive/Umojack/Riders.csv')
ss = pd.read_csv('/content/drive/My Drive/Umojack/SampleSubmission.csv')

In [6]:
# check data shapes
train.shape, test.shape, riders.shape, ss.shape

((179867, 21), (76791, 20), (2632, 4), (76791, 2))

In [7]:
# Preview train
train.head()

,ID,order_id,dispatch_day,dispatch_day_of_week,dispatch_time,client_id,client_type,order_license_status,order_carrier_type,vendor_type,rider_id,rider_license_status,rider_carrier_type,rider_amount,rider_lat,rider_long,pickup_lat,pickup_long,drop_off_lat,drop_off_long,target
0,ID_SCUW21PVAU,4435,27,6,09:02:54,593630,Business,0,2,Bike,30153,0,1,1080,-42.698343,-17.228539,-42.692371,-17.248305,-42.687442,-17.424682,1
1,ID_2HA7X30JMN,32711,30,7,13:01:37,837729,Personal,0,1,Bike,20884,0,1,730,-42.787317,-17.288252,-42.784046,-17.290121,-42.673267,-17.234595,2
2,ID_IAJWDTBY6M,8712,14,2,10:01:00,695129,Personal,0,2,Bike,33143,1,1,490,-42.749180,-17.287848,-42.765204,-17.293784,-42.813953,-17.294805,1
3,ID_LKSVPNYMTR,44869,22,3,14:11:16,1504660,Personal,0,2,Bike,96531,1,1,510,-42.836266,-17.311920,-42.831913,-17.315311,-42.812409,-17.265441,2
4,ID_O7N8Y918YH,57590,27,5,16:11:38,36869,Business,0,2,Bike,103546,0,0,400,-42.828195,-17.322818,-42.836056,-17.318111,-42.828517,-17.302052,0


In [8]:
# Preview riders
riders.head()

,Rider ID,Active Rider Age,Average Partner Rating,Number of Ratings
0,16261,308,21.05,321
1,8832,224,10.00,27
2,53866,238,17.76,25
3,46368,343,24.56,320
4,45609,399,14.97,214


In [9]:
# Merge rider dataset to train and test sets
train = train.merge(riders, how = 'left', left_on='rider_id', right_on='Rider ID')
test = test.merge(riders, how = 'left', left_on='rider_id', right_on='Rider ID')

# Preview merged dataframe
train.head()

,ID,order_id,dispatch_day,dispatch_day_of_week,dispatch_time,client_id,client_type,order_license_status,order_carrier_type,vendor_type,rider_id,rider_license_status,rider_carrier_type,rider_amount,rider_lat,rider_long,pickup_lat,pickup_long,drop_off_lat,drop_off_long,target,Rider ID,Active Rider Age,Average Partner Rating,Number of Ratings
0,ID_SCUW21PVAU,4435,27,6,09:02:54,593630,Business,0,2,Bike,30153,0,1,1080,-42.698343,-17.228539,-42.692371,-17.248305,-42.687442,-17.424682,1,30153,11,10.00,1
1,ID_2HA7X30JMN,32711,30,7,13:01:37,837729,Personal,0,1,Bike,20884,0,1,730,-42.787317,-17.288252,-42.784046,-17.290121,-42.673267,-17.234595,2,20884,68,24.13,229
2,ID_IAJWDTBY6M,8712,14,2,10:01:00,695129,Personal,0,2,Bike,33143,1,1,490,-42.749180,-17.287848,-42.765204,-17.293784,-42.813953,-17.294805,1,33143,273,24.92,123
3,ID_LKSVPNYMTR,44869,22,3,14:11:16,1504660,Personal,0,2,Bike,96531,1,1,510,-42.836266,-17.311920,-42.831913,-17.315311,-42.812409,-17.265441,2,96531,168,23.76,175
4,ID_O7N8Y918YH,57590,27,5,16:11:38,36869,Business,0,2,Bike,103546,0,0,400,-42.828195,-17.322818,-42.836056,-17.318111,-42.828517,-17.302052,0,103546,95,24.53,42


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179867 entries, 0 to 179866
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      179867 non-null  object 
 1   order_id                179867 non-null  int64  
 2   dispatch_day            179867 non-null  int64  
 3   dispatch_day_of_week    179867 non-null  int64  
 4   dispatch_time           179867 non-null  object 
 5   client_id               179867 non-null  int64  
 6   client_type             179867 non-null  object 
 7   order_license_status    179867 non-null  int64  
 8   order_carrier_type      179867 non-null  int64  
 9   vendor_type             179867 non-null  object 
 10  rider_id                179867 non-null  int64  
 11  rider_license_status    179867 non-null  int64  
 12  rider_carrier_type      179867 non-null  int64  
 13  rider_amount            179867 non-null  int64  
 14  rider_lat           

In [11]:
train["client_type"].value_counts()

Business    145902
Personal     33965
Name: client_type, dtype: int64

In [12]:
train["vendor_type"].value_counts()

Bike    179867
Name: vendor_type, dtype: int64

In [13]:
# One hot encoding
train = pd.get_dummies(train, columns=['client_type', 'vendor_type'], drop_first=True)
test = pd.get_dummies(test, columns=['client_type', 'vendor_type'], drop_first= True)

In [14]:
train.head()

,ID,order_id,dispatch_day,dispatch_day_of_week,dispatch_time,client_id,order_license_status,order_carrier_type,rider_id,rider_license_status,rider_carrier_type,rider_amount,rider_lat,rider_long,pickup_lat,pickup_long,drop_off_lat,drop_off_long,target,Rider ID,Active Rider Age,Average Partner Rating,Number of Ratings,client_type_Personal
0,ID_SCUW21PVAU,4435,27,6,09:02:54,593630,0,2,30153,0,1,1080,-42.698343,-17.228539,-42.692371,-17.248305,-42.687442,-17.424682,1,30153,11,10.00,1,0
1,ID_2HA7X30JMN,32711,30,7,13:01:37,837729,0,1,20884,0,1,730,-42.787317,-17.288252,-42.784046,-17.290121,-42.673267,-17.234595,2,20884,68,24.13,229,1
2,ID_IAJWDTBY6M,8712,14,2,10:01:00,695129,0,2,33143,1,1,490,-42.749180,-17.287848,-42.765204,-17.293784,-42.813953,-17.294805,1,33143,273,24.92,123,1
3,ID_LKSVPNYMTR,44869,22,3,14:11:16,1504660,0,2,96531,1,1,510,-42.836266,-17.311920,-42.831913,-17.315311,-42.812409,-17.265441,2,96531,168,23.76,175,1
4,ID_O7N8Y918YH,57590,27,5,16:11:38,36869,0,2,103546,0,0,400,-42.828195,-17.322818,-42.836056,-17.318111,-42.828517,-17.302052,0,103546,95,24.53,42,0


In [15]:
test.head()

,ID,order_id,dispatch_day,dispatch_day_of_week,dispatch_time,client_id,order_license_status,order_carrier_type,rider_id,rider_license_status,rider_carrier_type,rider_amount,rider_lat,rider_long,pickup_lat,pickup_long,drop_off_lat,drop_off_long,Rider ID,Active Rider Age,Average Partner Rating,Number of Ratings,client_type_Personal
0,ID_3B4D2Q2DSI,28636,2,6,12:10:59,593630,0,2,57224,1,1,450,-42.912776,-17.265738,-42.881861,-17.280865,-42.880476,-17.230240,57224,175,23.04,15,0
1,ID_7MPWFJ9XFI,60465,11,4,17:10:03,2378775,1,2,106329,1,1,560,-42.831932,-17.284140,-42.829545,-17.279453,-42.851920,-17.314558,106329,8,19.34,332,1
2,ID_5VVT3Q3M5B,25249,30,3,12:02:01,2121681,0,2,90551,0,0,450,-42.806437,-17.285166,-42.810915,-17.291086,-42.807165,-17.253860,90551,105,10.00,3,1
3,ID_C2GTVS1H7K,21224,27,2,11:10:32,38801,0,2,90482,1,1,440,-42.828448,-17.301909,-42.828517,-17.302052,-42.795516,-17.287921,90482,38,24.76,83,0
4,ID_0YGC8V3PFT,58287,14,6,16:12:55,153111,0,2,101959,1,1,500,-42.808912,-17.259955,-42.805154,-17.262939,-42.706475,-17.257454,101959,167,9.15,16,0


In [16]:
train_cols = train.columns.difference(['ID', 'order_id', 'rider_id', 'Rider ID','client_id'])
test_cols = test.columns.difference(['ID', 'order_id', 'rider_id', 'Rider ID','client_id']) 

In [17]:
train1 = train[train_cols]
test1 = test[test_cols]
train1.head()


,Active Rider Age,Average Partner Rating,Number of Ratings,client_type_Personal,dispatch_day,dispatch_day_of_week,dispatch_time,drop_off_lat,drop_off_long,order_carrier_type,order_license_status,pickup_lat,pickup_long,rider_amount,rider_carrier_type,rider_lat,rider_license_status,rider_long,target
0,11,10.00,1,0,27,6,09:02:54,-42.687442,-17.424682,2,0,-42.692371,-17.248305,1080,1,-42.698343,0,-17.228539,1
1,68,24.13,229,1,30,7,13:01:37,-42.673267,-17.234595,1,0,-42.784046,-17.290121,730,1,-42.787317,0,-17.288252,2
2,273,24.92,123,1,14,2,10:01:00,-42.813953,-17.294805,2,0,-42.765204,-17.293784,490,1,-42.749180,1,-17.287848,1
3,168,23.76,175,1,22,3,14:11:16,-42.812409,-17.265441,2,0,-42.831913,-17.315311,510,1,-42.836266,1,-17.311920,2
4,95,24.53,42,0,27,5,16:11:38,-42.828517,-17.302052,2,0,-42.836056,-17.318111,400,0,-42.828195,0,-17.322818,0


In [18]:
test1.head()

,Active Rider Age,Average Partner Rating,Number of Ratings,client_type_Personal,dispatch_day,dispatch_day_of_week,dispatch_time,drop_off_lat,drop_off_long,order_carrier_type,order_license_status,pickup_lat,pickup_long,rider_amount,rider_carrier_type,rider_lat,rider_license_status,rider_long
0,175,23.04,15,0,2,6,12:10:59,-42.880476,-17.230240,2,0,-42.881861,-17.280865,450,1,-42.912776,1,-17.265738
1,8,19.34,332,1,11,4,17:10:03,-42.851920,-17.314558,2,1,-42.829545,-17.279453,560,1,-42.831932,1,-17.284140
2,105,10.00,3,1,30,3,12:02:01,-42.807165,-17.253860,2,0,-42.810915,-17.291086,450,0,-42.806437,0,-17.285166
3,38,24.76,83,0,27,2,11:10:32,-42.795516,-17.287921,2,0,-42.828517,-17.302052,440,1,-42.828448,1,-17.301909
4,167,9.15,16,0,14,6,16:12:55,-42.706475,-17.257454,2,0,-42.805154,-17.262939,500,1,-42.808912,1,-17.259955


In [19]:
# changing the dispatch time into hour, min and seconds

train1["dispatch_time"]= pd.to_datetime(train1["dispatch_time"], format='%H:%M:%S')

train1["dispatch_time_hour"] = train1["dispatch_time"].dt.hour
train1["dispatch_time_min"] = train1["dispatch_time"].dt.minute
train1["dispatch_time_secs"] = train1["dispatch_time"].dt.second


test1["dispatch_time"]= pd.to_datetime(test1["dispatch_time"], format='%H:%M:%S')

test1["dispatch_time_hour"] = test1["dispatch_time"].dt.hour
test1["dispatch_time_min"] = test1["dispatch_time"].dt.minute
test1["dispatch_time_secs"] = test1["dispatch_time"].dt.second


In [21]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179867 entries, 0 to 179866
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Active Rider Age        179867 non-null  int64         
 1   Average Partner Rating  179867 non-null  float64       
 2   Number of Ratings       179867 non-null  int64         
 3   client_type_Personal    179867 non-null  uint8         
 4   dispatch_day            179867 non-null  int64         
 5   dispatch_day_of_week    179867 non-null  int64         
 6   dispatch_time           179867 non-null  datetime64[ns]
 7   drop_off_lat            179867 non-null  float64       
 8   drop_off_long           179867 non-null  float64       
 9   order_carrier_type      179867 non-null  int64         
 10  order_license_status    179867 non-null  int64         
 11  pickup_lat              179867 non-null  float64       
 12  pickup_long             179867

In [22]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76791 entries, 0 to 76790
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Active Rider Age        76791 non-null  int64         
 1   Average Partner Rating  76791 non-null  float64       
 2   Number of Ratings       76791 non-null  int64         
 3   client_type_Personal    76791 non-null  uint8         
 4   dispatch_day            76791 non-null  int64         
 5   dispatch_day_of_week    76791 non-null  int64         
 6   dispatch_time           76791 non-null  datetime64[ns]
 7   drop_off_lat            76791 non-null  float64       
 8   drop_off_long           76791 non-null  float64       
 9   order_carrier_type      76791 non-null  int64         
 10  order_license_status    76791 non-null  int64         
 11  pickup_lat              76791 non-null  float64       
 12  pickup_long             76791 non-null  float6

In [23]:
train1.head()

,Active Rider Age,Average Partner Rating,Number of Ratings,client_type_Personal,dispatch_day,dispatch_day_of_week,dispatch_time,drop_off_lat,drop_off_long,order_carrier_type,order_license_status,pickup_lat,pickup_long,rider_amount,rider_carrier_type,rider_lat,rider_license_status,rider_long,target,dispatch_time_hour,dispatch_time_min,dispatch_time_secs
0,11,10.00,1,0,27,6,1900-01-01 09:02:54,-42.687442,-17.424682,2,0,-42.692371,-17.248305,1080,1,-42.698343,0,-17.228539,1,9,2,54
1,68,24.13,229,1,30,7,1900-01-01 13:01:37,-42.673267,-17.234595,1,0,-42.784046,-17.290121,730,1,-42.787317,0,-17.288252,2,13,1,37
2,273,24.92,123,1,14,2,1900-01-01 10:01:00,-42.813953,-17.294805,2,0,-42.765204,-17.293784,490,1,-42.749180,1,-17.287848,1,10,1,0
3,168,23.76,175,1,22,3,1900-01-01 14:11:16,-42.812409,-17.265441,2,0,-42.831913,-17.315311,510,1,-42.836266,1,-17.311920,2,14,11,16
4,95,24.53,42,0,27,5,1900-01-01 16:11:38,-42.828517,-17.302052,2,0,-42.836056,-17.318111,400,0,-42.828195,0,-17.322818,0,16,11,38


In [24]:
# calculating distance of longitude and latitude of Train Pickup and drop_off

distances = []

for i in range(len(train1)):
    R = 6373.0 #radius of the Earth


    lat1 = math.radians(train1["pickup_lat"][i]) #coordinates
    lon1 = math.radians(train1["pickup_long"][i])

    lat2 = math.radians(train1["drop_off_lat"][i])
    lon2 = math.radians(train1["drop_off_long"][i])

    dlon = lon2 - lon1 #change in coordinates
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2 #Haversine formula

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distances.append(R * c)


train1["distance"]= distances
train1 = train1.drop(["pickup_lat","pickup_long","drop_off_lat","drop_off_long", "dispatch_time"], 1) # "dispatch_time", "vendor_type_Bike"
   

In [25]:
# calculating distance of longitude and latitude of Test Pickup and drop_off

distances_test = []

for i in range(len(test1)):
    R = 6373.0 #radius of the Earth


    lat1 = math.radians(test1["pickup_lat"][i]) #coordinates
    lon1 = math.radians(test1["pickup_long"][i])

    lat2 = math.radians(test1["drop_off_lat"][i])
    lon2 = math.radians(test1["drop_off_long"][i])

    dlon = lon2 - lon1 #change in coordinates
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2 #Haversine formula

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distances_test.append(R * c)

test1["distance"] = distances_test
test1 = test1.drop(["pickup_lat","pickup_long","drop_off_lat","drop_off_long", "dispatch_time"], 1) #"dispatch_time",  "vendor_type_Bike"

In [26]:
# # Feature binning
# from sklearn.preprocessing import KBinsDiscretizer
# age_discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')

# train1['Age_Bins'] =age_discretizer.fit_transform(train1['Active Rider Age'].values.reshape(-1,1)).astype(int)
# test1['Age_Bins'] =age_discretizer.fit_transform(test1['Active Rider Age'].values.reshape(-1,1)).astype(int)

# train1 = train1.drop("Active Rider Age", 1)
# test1 = test1.drop("Active Rider Age", 1)

In [35]:
train1.head()

,Active Rider Age,Average Partner Rating,Number of Ratings,client_type_Personal,dispatch_day,dispatch_day_of_week,order_carrier_type,order_license_status,rider_amount,rider_carrier_type,rider_lat,rider_license_status,rider_long,target,dispatch_time_hour,dispatch_time_min,dispatch_time_secs,distance
0,11,10.00,1,0,27,6,2,0,1080,1,-42.698343,0,-17.228539,1,9,2,54,14.430604
1,68,24.13,229,1,30,7,1,0,730,1,-42.787317,0,-17.288252,2,13,1,37,13.130575
2,273,24.92,123,1,14,2,2,0,490,1,-42.749180,1,-17.287848,1,10,1,0,5.423039
3,168,23.76,175,1,22,3,2,0,510,1,-42.836266,1,-17.311920,2,14,11,16,4.610758
4,95,24.53,42,0,27,5,2,0,400,0,-42.828195,0,-17.322818,0,16,11,38,1.555388


In [27]:

X = train1.drop("target", 1)
y = train1.target



In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3031)

In [38]:
# # Train a model using LgbmClassifier

model = LGBMClassifier(random_state=3031)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Check score
accuracy_score(y_test, y_pred)

In [39]:
# using xgboost


xgb = XGBClassifier(random_state=1)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)


# Check score
accuracy_score(y_test,xgb_pred)

In [28]:
#  Using CatBoostClassifier


cat = CatBoostClassifier(random_state=1)
cat.fit(X_train, y_train)
cat_pred = cat.predict(X_test)


# Check score
accuracy_score(y_test,cat_pred)

In [29]:
# fitting the whole train dataset

cat.fit(X,y)

# Make predictions in test set and prepare submission file
pred_cat = cat.predict(test1)
sub_file = ss.copy()
sub_file.target = pred_cat
sub_file.to_csv('Baseline7.csv', index = False)

In [37]:

predictions = []

fold=StratifiedKFold(n_splits=25,shuffle=True, random_state=42)

for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    classifier2=LGBMClassifier(learning_rate=0.1, n_estimators=800, scale_pos_weight = 5.0)  #, ,

    classifier2.fit(X_train, Y_train)
    preds=classifier2.predict(X_test)
    predictions.append(classifier2.predict(test1))

In [38]:

predictions = pd.DataFrame(predictions)

In [39]:
prediction1 = predictions.copy()

In [41]:

prediction1 = prediction1.mode().T
prediction1

,0,1
0,0.0,NaN
1,0.0,NaN
2,0.0,NaN
3,0.0,NaN
4,0.0,NaN
...,...,...
76786,0.0,NaN
76787,0.0,NaN
76788,0.0,NaN
76789,1.0,NaN


In [43]:
sub = prediction1[0].astype(int)

sub_file = ss.copy()

sub_file.target = sub
sub_file.to_csv('sub.csv', index = False)